In [1]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv(override=True)

True

https://github.com/Coding-Crashkurse/LangChain-Custom-Loaders/blob/main/loaders.ipynb

In [84]:
# model_name = 'gpt-3.5-turbo-16k-0613'
model_name = 'gpt-4-32k'

In [85]:
import tiktoken

def concatenate_page_contents(input_documents, separator):
    page_contents = [doc.dict()['page_content'] for doc in input_documents]
    return separator.join(page_contents)

def num_tokens_from_string(string: str, model_name: str = model_name) -> int:
    encoding = tiktoken.encoding_for_model(model_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

In [86]:
from langchain.document_loaders import DirectoryLoader

loader = DirectoryLoader(
    '../../data/ndis', # my local directory
    glob='**/*.pdf',     # we only get pdfs
    show_progress=True
)
defaultLoaderDocs = loader.load()
defaultLoaderDocs

100%|██████████| 8/8 [01:12<00:00,  9.04s/it]


[Document(page_content='National Disability Insurance Scheme\n\nPricing Arrangements for Specialist Disability Accommodation 2023-24\n\nValid from: 1 July 2023 Version 1.2\n\n(Released 12 July 2023)\n\nCopyright\n\n© National Disability Insurance Agency 2023\n\nUse of National Disability Insurance Agency copyright material\n\nThe material in this report with the exception of logos, trademarks, third party material and other content as specified is licensed under Creative Commons CC NC licence, version 4.0. With the exception of logos, trademarks, third party material and other content as specified, you may reproduce the material in this NDIS Pricing Arrangements for Specialist Disability Accommodation, provided you acknowledge the National Disability Insurance Agency as the owner of all intellectual property rights in the reproduced material by using ‘© National Disability Insurance Agency 2023’ and do not use the material for commercial purposes.\n\nReproduction of any Creative Common

In [14]:
type(docs[0])

<class 'langchain.schema.document.Document'>

In [26]:
# THE GOOD ONE
#--------------------------

from datetime import datetime
from typing import List, Optional, Dict
from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader
import pdfplumber

# Implementing the PDFLoader
class MyPDFLoader(BaseLoader):

    def __init__(
        self,
        file_path: str,
        pages_per_document: int = 1,
        encoding: Optional[str] = None
    ):
        self.file_path = file_path
        self.pages_per_document = int(pages_per_document)
        self.encoding = encoding

    def extract_interleaved_text_and_tables(self, page) -> str:
        """Extract interleaved text and tables from a given page."""
        full_text = page.extract_text()
        tables = page.extract_tables()

        table_texts = []
        for table in tables:
            table_text = "\n".join(["\t".join([cell if cell is not None else "" for cell in row]) for row in table])
            table_texts.append(table_text)

        processed_tables = set()
        for table_text in table_texts:
            first_cell_content = table_text.split("\n")[0].split("\t")[0]
            if first_cell_content not in processed_tables:
                insertion_index = full_text.rfind(first_cell_content)
                if insertion_index != -1:
                    full_text = full_text.replace(first_cell_content, table_text, 1)
                    processed_tables.add(first_cell_content)
                
        return full_text

    def load(self) -> List[Document]:
        docs = []
        with pdfplumber.open(self.file_path) as pdf:
            total_pages = len(pdf.pages)
            for start_page in range(0, total_pages, self.pages_per_document):
                end_page = min(start_page + self.pages_per_document, total_pages)
                page_contents = []
                for page_number in range(start_page, end_page):
                    page_content = self.extract_interleaved_text_and_tables(pdf.pages[page_number])
                    page_contents.append(page_content)
                
                content = "\n".join(page_contents)
                timestamp = datetime.now().isoformat()
                metadata = {
                    "title": pdf.metadata["Title"],
                    "source": self.file_path,
                    "timestamp": timestamp,
                    "start_page": start_page + 1,  # +1 for 1-based page numbering
                    "end_page": end_page
                }
                doc = Document(page_content=content, metadata=metadata)
                docs.append(doc)

        return docs




In [145]:
filename = '../../data/ndis2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf'
filepath = '../../data/ndis2'

In [ ]:
# Testing the new PDFLoader with 5 pages per Document
loader = MyPDFLoader(filename, pages_per_document=5)
documents = loader.load()
documents[0].metadata  # Displaying metadata of the first Document for a preview

In [147]:
#https://api.python.langchain.com/en/latest/document_loaders/langchain.document_loaders.pdf.PDFPlumberLoader.html#langchain.document_loaders.pdf.PDFPlumberLoader.load

from langchain.document_loaders.pdf import PDFPlumberLoader

loader = PDFPlumberLoader(filename)
plumberDocs = loader.load()
plumberDocs[0].metadata

{'source': '../../data/ndis2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'file_path': '../../data/ndis2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'page': 1, 'total_pages': 101, 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Enabled': 'true', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_SetDate': '2022-02-24T21:20:38Z', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Method': 'Privileged', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_Name': 'OFFICIAL', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_SiteId': 'cd778b65-752d-454a-87cf-b9990fe58993', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_ActionId': 'a41679f1-751c-43a0-a310-7842fbe5f727', 'MSIP_Label_2b83f8d7-e91f-4eee-a336-52a8061c0503_ContentBits': '0', 'Title': 'NDIS Pricing Arrangements and Price Limits 2023-24', 'Author': 'Rundle, Vincent', 'Creator': 'Microsoft® Word for Microsoft 365', 'CreationDate': "D:20230615172130+10'00'", 'ModDate': "D:20230615172130+10'00'", 'Producer'

In [152]:
from pprint import pprint
pprint(plumberDocs[25].page_content)

('General Claiming Rules\n'
 'Example – Activity Based Transport\n'
 'A provider has agreed to deliver a support in the Assistance with Social, '
 'Economic and Community Participation\n'
 'Support Category to a participant and the participant has requested that the '
 'provider transport the participant from\n'
 'their home to a local swimming pool and back again, as part of that '
 'support.\n'
 'The transport by a non-disability modified vehicle takes 25 minutes to '
 'arrive at the swimming pool, including the time\n'
 'to assist the participant to and from the vehicle and getting them set up to '
 'participate in the activity. The support\n'
 'worker accompanies the participant in the vehicle. They then provide 40 '
 'minutes of support to the participant at the\n'
 'pool. Afterwards, they spend 20 minutes returning the participant to their '
 'home by the same vehicle. The pool is\n'
 '10 kilometres from the participant’s home.\n'
 'The provider and participant have agreed an hou

In [167]:
import os
from langchain.document_loaders import CSVLoader

class MyDirectoryLoader:

    def __init__(self, dir_path):
        self.dir_path = dir_path
        
    def check_args(self):
        print(**self.pdf_args.keys())

    def load(self):
        docs = []
        for root, _, files in os.walk(self.dir_path):
            for file in files:
                print('file:', file)
                file_path = os.path.join(root, file)
                if file_path.endswith('.csv'):
                    loader = CSVLoader(file_path)
                elif file_path.endswith('.pdf'):
                    # pages_per_document=self.kwargs.get('pages_per_document', 1)
                    # loader = MyPDFLoader(file_path, pages_per_document=pages_per_document)
                    loader = PDFPlumberLoader(file_path)
                else:
                    print(f"Do not process the file: {file_path}")
                    continue
                loaded_docs = loader.load()
                docs.extend(loaded_docs)
        return docs

In [168]:
loader = MyDirectoryLoader(os.path.join(os.getcwd(), "../../data/ndis2"), pages_per_document=5 )
docs = loader.load()
docs

TypeError: MyDirectoryLoader.__init__() got an unexpected keyword argument 'pages_per_document'

In [157]:
for t in docs[10:20]:
    # print('metadata:', t.metadata)
    print('doc length:', len(t.page_content))
    print('num_token:', num_tokens_from_string(t.page_content))

doc length: 2585
num_token: 517
doc length: 2799
num_token: 572
doc length: 1266
num_token: 260
doc length: 3045
num_token: 589
doc length: 3265
num_token: 743
doc length: 3117
num_token: 706
doc length: 2987
num_token: 601
doc length: 3046
num_token: 630
doc length: 3701
num_token: 773
doc length: 3761
num_token: 1286


## text splitter

In [159]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
texts = splitter.split_documents(docs)
for t in texts[10:20]:
#     print('metadata:', t.metadata['source'])
    print('text length:', len(t.page_content))
    print('num_token:', num_tokens_from_string(t.page_content))
#     #print('text:', t.page_content)

text length: 966
num_token: 92
text length: 1015
num_token: 98
text length: 978
num_token: 98
text length: 334
num_token: 61
text length: 908
num_token: 113
text length: 983
num_token: 91
text length: 888
num_token: 81
text length: 915
num_token: 113
text length: 914
num_token: 98
text length: 325
num_token: 61


## Retrieve and delete existing collection (optional)

In [46]:
# Check existing colletion in chroma (https://docs.trychroma.com/usage-guide)

from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings
from chromadb import Client

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )
chromaClient = Client(client_settings)
coll = chromaClient.get_collection(name='NDIS_COLLECTION', embedding_function=OpenAIEmbeddings())
coll.count()



In [50]:
chromaClient.delete_collection(name='NDIS_COLLECTION')

## indexing


In [160]:
print(model_name)

gpt-4-32k


In [164]:
# choose collection

#collection_name = 'NDIS_COLLECTION' # 5 pages per documents, text splitter size 4096 overlap 200
#collection_name = 'NDIS_COLL_PAGES_5_TEXTS_1024_256' # 5 pages per documents, text splitter size 1024 overlap 256
# collection_name = 'NDIS_COLL_PAGES_5_TEXTS_512_128' # 5 pages per document, text splitter size 512 overlap 128
collection_name = 'NDIS_PDFPLUMBER_1_TEXTS_1024_128'

In [165]:
from chromadb.config import Settings
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

db = Chroma.from_documents(texts, client_settings=client_settings, embedding = OpenAIEmbeddings(), collection_name=collection_name)


# Index search

In [166]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings

client_settings = Settings(
        chroma_api_impl="rest",
        chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
        chroma_server_http_port="8000"
    )

chromaDb = Chroma(collection_name=collection_name, 
                     embedding_function=OpenAIEmbeddings(),
                     client_settings=client_settings)

In [95]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo-16k')

retriever = chromaDb.as_retriever(search_kwargs={"k": 10})

qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type='stuff',
    retriever=retriever,
    # chain_type_kwargs={"prompt": PROMPT}, 
    # return_source_documents=True,
    verbose=True
)

### 'NDIS_COLL_PAGES_5_TEXTS_512_128' 5 pages per documents, text splitter size 512 overlap 128 with k=1

In [96]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [97]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

### 'NDIS_COLL_PAGES_5_TEXTS_512_128' 5 pages per documents, text splitter size 512 overlap 128 with k=3

In [82]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [83]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 17867 tokens. Please reduce the length of the messages.

### 'NDIS_COLL_PAGES_5_TEXTS_1024_256' with k = 2

In [75]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

In [76]:
query = "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': "I'm using item code 01_019_0120_1_1 - House or Yard Maintenance. to invoice a customer for my mowing service, can I invoice the travel time too? what item code for travel time?"}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides 

InvalidRequestError: This model's maximum context length is 16385 tokens. However, your messages resulted in 17867 tokens. Please reduce the length of the messages.

### NDIS_COLLECTION with k = 2

In [58]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"
# result = qa.run(query)
result = qa({"query": query})

pprint(result)

Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW'}
Count Inputs: 1


> Entering new RetrievalQA chain...
Chroma Similarity search: [(Document(page_content='Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-

# Doing my own similarity search and chain

In [135]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo-16k')

template = """
You are a helpful, polite and well-mannered bot, a specialist in the NDIS Price Guide shared below.
You are striving to help providers to invoice for their services or the items they have sold to the participant.
I will share a provider's query with you - to select the right item code and determine the max price, you most generally need to know:
- the nature of the service or item sold
- the location the service was provided (postcode)
- the duration of service or the quantity of items sold
- the time of day if service (start/end)
Upon receiving the user query and the price guide context, your aim is to:
- select for them the approriate item code from the price guide
- determine the maximum price they can charge for the good or service
- more generally, by advising them following recommendations set up in the price guide for that particular service if any
When replying, you will follow ALL of the rules below:

1/ If some information is missing to determine what item code to use (location, quantity, remoteness, time of day etc), please ask that information to the user
2/ If you don't have enough information to answer the user query, don't invent anything and say you don't know

Provider query:
{query}

Here are the relevant extracts from the price guide:
{price_guide_context}

Please write the most informative answer to the provider query:
"""

prompt = PromptTemplate(
    input_variables=['query', 'price_guide_context'],
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)

In [136]:

def similarity_search2(query, db):
    similar_vectors = db.similarity_search(query, k=3)

    page_contents = [doc.page_content for doc in similar_response]

    # print(page_contents_array)

    return page_contents_array

In [137]:
from langchain.embeddings import OpenAIEmbeddings
from chromadb.config import Settings
from chromadb import Client

def get_chroma_collection(collection_name):

    client_settings = Settings(
            chroma_api_impl="rest",
            chroma_server_host="host.docker.internal",  # when you run this inside a devcontainer you need to explicitely say host.docker.internal to signify "devcontainer host localhost"
            chroma_server_http_port="8000"
        )
    chromaClient = Client(client_settings)
    coll = chromaClient.get_collection(name=collection_name, embedding_function=OpenAIEmbeddings().embed_documents)
    # coll = chromaClient.get_collection(name='LANTERNPAY_COLLECTION', embedding_function=OpenAIEmbeddings().embed_documents)
    return coll

def similarity_search(query, coll, n_results=2):
    results = coll.query(query_texts=[query], n_results=2)
    metadatas = [ met for met in results['metadatas'][0]]
    docs = [ doc for doc in results['documents'][0]]
    return { 'documents': docs, 'metadatas': metadatas}

def get_query_response(query, n_results=2):
    similar_docs = similarity_search(
        query, 
        get_chroma_collection(collection_name), 
        n_results=2)
    response = chain.run(query=query, price_guide_context=similar_docs['documents'])
    return response
    

In [99]:
collection_name

'NDIS_COLL_PAGES_5_TEXTS_512_128'

In [125]:
query = "What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW"

In [138]:

response = get_query_response(query, n_results=2)


Inputs: {'query': 'What item code should I use when invoicing for a participant? I have mowed the lawn of the participant Saturday 5/08/3023 from 9 to 11 am in Pennant Hills, NSW', 'price_guide_context': ['Core – Assistance with Daily Life\nProviders of this support can also claim for the costs of:\n• Provider Travel – Non-Labour Costs using support item 01_799_0107_1_1.\nThis support item is subject to price limits as set out in the following Table.\nItem Number\t\t\tItem Name and Notes\t\t\tUnit\t\t\tNational\t\t\tRemote\t\t\t\tVery\t\n\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tRemote\t\n\t01_004_0107_1_1\t\t\tAssistance with Personal Domestic Activities\t\t\tHour\t\t\t$55.03\t\t\t$77.04\t\t\t$82.55\t Item Name and Notes Very\nUnit National Remote\nRemote\n01_004_0107_1_1 Assistance with Personal Domestic Activities Hour $55.03 $77.04 $82.55\nOn-Call Overnight Monitoring\nThis support item provides for overnight on-call assistance (either onsite or off-site) with, or\nsupervision of, personal t

'To invoice for mowing the lawn of the participant, you can use the item code 01_019_0120_1_1 - House or Yard Maintenance. This item code covers performing essential house and/or yard activities that the participant is not able to undertake. The unit for this item code is an hour. The maximum price you can charge for this service depends on the location. In Pennant Hills, NSW, the maximum price for this service is $79.64. \n\nPlease note that the price guide also provides information on claiming for provider travel. If you incurred any travel costs while providing the service, you may be able to claim for those as well. The specific item code for provider travel - non-labour costs is 01_799_0120_1_1. However, please ensure that the conditions for claiming provider travel are met as outlined in the price guide.\n\nIf you have any further questions or need additional information, please let me know.'

In [139]:
print(response)

To invoice for mowing the lawn of the participant, you can use the item code 01_019_0120_1_1 - House or Yard Maintenance. This item code covers performing essential house and/or yard activities that the participant is not able to undertake. The unit for this item code is an hour. The maximum price you can charge for this service depends on the location. In Pennant Hills, NSW, the maximum price for this service is $79.64. 

Please note that the price guide also provides information on claiming for provider travel. If you incurred any travel costs while providing the service, you may be able to claim for those as well. The specific item code for provider travel - non-labour costs is 01_799_0120_1_1. However, please ensure that the conditions for claiming provider travel are met as outlined in the price guide.

If you have any further questions or need additional information, please let me know.


In [127]:
similar_docs.keys()

dict_keys(['documents', 'metadatas'])

In [122]:
# print(similar_docs['documents'][0][1])
print(similar_docs['metadatas'][0][0])

{'title': 'NDIS Pricing Arrangements and Price Limits 2023-24', 'source': '/workspaces/IsabelleLangchain/src/ndis/../../data/ndis2/PB NDIS Pricing Arrangements and Price Limits 2023-24 .pdf', 'timestamp': '2023-08-06T09:01:46.122650', 'start_page': 41, 'end_page': 45}
